<a href="https://colab.research.google.com/github/Athul-Finsurge/French-translator/blob/main/tmp_9976zahYdP2KaxtK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import pandas as pd
import gzip
import codecs
import re
import time
import tensorflow as tf
import numpy as np

In [47]:
frdata=[]
endata=[]
with open(r'/content/train_fr_lines.txt',encoding='utf-8') as frfile:
    for li in frfile:
        frdata.append(li)
with open(r'/content/train_en_lines.txt',encoding='utf-8') as enfile:
    for li in enfile:
        endata.append(li)
mtdata = pd.DataFrame({'FR':frdata,'EN':endata})
mtdata['FR_len'] = mtdata['FR'].apply(lambda x: len(x.split(' ')))
mtdata['EN_len'] = mtdata['EN'].apply(lambda x: len(x.split(' ')))
mtdata['FR_len'].sort_values()

210996      1
104793      1
104266      1
104255      1
104122      1
         ... 
195666    466
176618    468
22223     504
194573    563
174898    638
Name: FR_len, Length: 210997, dtype: int64

In [48]:
mtdata.iloc[174898]
mtdata = mtdata.loc[mtdata['FR_len']<8]
mtdata['FR_len'].sort_values()

210996    1
73337     1
22788     1
44134     1
139312    1
         ..
80020     7
80010     7
80007     7
80063     7
0         7
Name: FR_len, Length: 40099, dtype: int64

In [49]:
mtdata_fr = []
for fr in mtdata.FR:
    mtdata_fr.append(fr)
mtdata_en = []
for en in mtdata.EN:
    mtdata_en.append(en)

In [50]:
# mtdata_fr

In [51]:
import tensorflow as tf
tf.version.VERSION
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time


In [52]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())
  # print(w)

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [53]:
def create_dataset(path, num_examples):
  # lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
  lines  = path
  # lines =[ val for val in lines if len(val)<50 ]
  # le = max(lines,key= len)
  # print(le,len(le))
  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]
  

  return word_pairs

In [54]:
# fr = create_dataset(r'C:\Users\Finsurge25\Downloads\train_fr_lines.txt', None)
# en = create_dataset(r'C:\Users\Finsurge25\Downloads\train_en_lines.txt', None)
fr = create_dataset(mtdata_fr,None)
en = create_dataset(mtdata_en,None)
fr =[ val[0] for val in fr]
en =[ val[0] for val in en]
le = max(fr,key= len)
print(le,len(le))
# le = max(en,key= len)
# print(le,len(le))
# print(le)
print(en[-1],len(en))
print(fr[-1],len(fr))

<start> translator href http www . ted . com profiles mohamed lukman bin mohamed syed translator <end> 102
<start> thank you . <end> 40099
<start> merci . <end> 40099


In [55]:
print(le,len(le))


<start> translator href http www . ted . com profiles mohamed lukman bin mohamed syed translator <end> 102


In [56]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,padding='post')

  return tensor, lang_tokenizer

In [57]:
def load_dataset(path, num_examples=None):
  # creating cleaned input, output pairs
  lang = create_dataset(path, num_examples)
  lang =[ val[0] for val in lang]
  print(lang)
  # print(lang)
  tensor, lang_tokenizer = tokenize(lang)
#   target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return tensor,lang_tokenizer

In [58]:
# Try experimenting with the size of that dataset
num_examples = 30000
input_tensor,inp_lang,  = load_dataset(mtdata_fr, num_examples)
target_tensor,targ_lang = load_dataset(mtdata_en,num_examples)
# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

['<start> voici bill lange . je suis dave gallo . <end>', '<start> la profondeur moyenne est environ , km . <end>', '<start> voici une meduse . <end>', '<start> mais voyez vous tous ces bras en mouvement ? <end>', '<start> c est un animal colonial . <end>', '<start> voici la meduse aile x de la mort . <end>', '<start> on ne les connait pas bien . <end>', '<start> c est comme un coq sous marin . <end>', '<start> voici un de mes preferes . quelle tete ! <end>', '<start> vous regardez tout simplement des donnees scientifiques . <end>', '<start> nous contournons une falaise ici a droite . <end>', '<start> voici une crete volcanique . <end>', '<start> nous zoomons de plus en plus . <end>', '<start> voici ce qu on appelle un conduit hydrothermal . <end>', '<start> voici un de ces conduits hydrothermaux . <end>', '<start> la pression ici est d environ bar . <end>', '<start> ceux ci sont des vers tubulaires . <end>', '<start> oh ! tres bien . <end>', '<start> ces vers vivent dans une temperatu

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [59]:
# embeddings_index = build_word_vector_matrix(r'C:\Users\Finsurge25\Downloads\glove.6B.50d.txt')


In [60]:
def build_word_vector_matrix(vector_file):
    embedding_index = {}
    with codecs.open(vector_file, 'r', 'utf-8') as f:
        for i, line in enumerate(f):
            sr = line.split()
            if(len(sr)<26):
                continue
            word = sr[0]
            embedding = np.asarray(sr[1:], dtype='float32')
            embedding_index[word] = embedding
    return embedding_index

In [61]:
embeddings_index = build_word_vector_matrix(r'/content/glove.6B.50d.txt')


In [62]:
def build_word2id_mapping(word_counts_dict):
    word2int = {} 
    count_threshold = 20
    value = 0
    for word, count in word_counts_dict.items():
        if count >= count_threshold or word in embeddings_index:
            word2int[word] = value
            value += 1
    print("Before special token",len(word2int))        
    special_codes = ["<start>","<end>"]   

    for code in special_codes:
        word2int[code] = len(word2int)
        print("After special token",len(word2int))

    int2word = {}
    for word, value in word2int.items():
        int2word[value] = word
    return word2int,int2word

In [63]:
# Creating training and validation sets using an 80-20 split
# frf =list(fre_word2int.keys())
# enf=list(eng_word2int.keys())
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

24000 24000 6000 6000


In [64]:
print("English Word2id mapping \n")
eng_word2int,eng_int2word = build_word2id_mapping(targ_lang.word_index)
print("\nFrench Word2id mapping\n ")
fre_word2int,fre_int2word = build_word2id_mapping(inp_lang.word_index)

English Word2id mapping 

Before special token 12024
After special token 12025
After special token 12026

French Word2id mapping
 
Before special token 15175
After special token 15176
After special token 15177


In [65]:
# inp_lang.word_index

In [66]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [67]:
max_len = np.max([len(input_tensor_train[i]) for i in range(len(input_tensor_train))])
print(max_len)

18


In [68]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
90 ----> ou
4 ----> est
58 ----> elle
10 ----> a
13 ----> l
603 ----> interieur
59 ----> du
14423 ----> neocortex
7 ----> ?
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
125 ----> where
9 ----> is
6 ----> it
440 ----> inside
5 ----> the
11491 ----> neocortex
13 ----> ?
2 ----> <end>


In [69]:
# inp_lang.word_index
input_tensor_train

array([[   1,   90,    4, ...,    0,    0,    0],
       [   1,   39,   74, ...,    0,    0,    0],
       [   1,   33, 4994, ...,    0,    0,    0],
       ...,
       [   1,   25, 3825, ...,    0,    0,    0],
       [   1,    6,    4, ...,    0,    0,    0],
       [   1,   15,   21, ...,    0,    0,    0]], dtype=int32)

In [70]:
def build_embeddings(word2int):
    embedding_dim = 50
    j=0
    nwords = len(word2int)
    word_emb_matrix = np.zeros((nwords, embedding_dim), dtype=np.float32)
    for word, i in word2int.items():
        if word in embeddings_index:
            word_emb_matrix[i] = embeddings_index[word]
        else:
            j +=1
            new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
            word_emb_matrix[i] = new_embedding
    print (j/nwords)
    return word_emb_matrix

In [71]:
BUFFER_SIZE = len(input_tensor_train)
print(BUFFER_SIZE)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 50
units = 1024
vocab_inp_size = len(inp_lang.word_index)
vocab_tar_size = len(targ_lang.word_index)
print(vocab_inp_size)
print(vocab_tar_size)

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

24000
15177
12026


In [72]:
fre_int2word

{0: '.',
 1: 'est',
 2: ',',
 3: 'c',
 4: '?',
 5: 'de',
 6: 'la',
 7: 'a',
 8: 'et',
 9: 'le',
 10: 'l',
 11: 'translator',
 12: 'reviewer',
 13: 'un',
 14: 'vous',
 15: 'ted',
 16: 'com',
 17: 'www',
 18: 'href',
 19: 'http',
 20: 'profiles',
 21: 'nous',
 22: 'il',
 23: 'je',
 24: 'les',
 25: 'une',
 26: 'ce',
 27: 'd',
 28: 'pas',
 29: 'en',
 30: 'ca',
 31: 'merci',
 32: 'que',
 33: 'j',
 34: 'des',
 35: 'n',
 36: 'ils',
 37: 'etait',
 38: 'mais',
 39: 'on',
 40: 'qu',
 41: 'donc',
 42: 'ai',
 43: 'ne',
 44: 'voici',
 45: 'sont',
 46: 'beaucoup',
 47: 'cela',
 48: 's',
 49: 'y',
 50: 'tres',
 51: 'bien',
 52: 'pour',
 53: 'tout',
 54: 'fait',
 55: 'elle',
 56: 'du',
 57: 'plus',
 58: 'alors',
 59: 'dans',
 60: 'qui',
 61: 'etre',
 62: 'cette',
 63: 'comment',
 64: 'avons',
 65: 'ont',
 66: 'vraiment',
 67: 'faire',
 68: 'sur',
 69: 'voila',
 70: 'ces',
 71: 'se',
 72: 'peut',
 73: '!',
 74: 'maintenant',
 75: 'elisabeth',
 76: 'buffard',
 77: 'moi',
 78: 'pourquoi',
 79: 'mon',
 80

In [73]:
example_input_batch, example_target_batch = next(iter(dataset))
# le = max(example_input_batch,key= len)
# print(le)
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 18]), TensorShape([64, 25]))

In [74]:
fr_embeddings_matrix = build_embeddings(fre_word2int)
en_embeddings_matrix = build_embeddings(eng_word2int)

0.6253541543124465
0.05222018958922335


In [75]:
print('Length of french embeddings:',len(fr_embeddings_matrix))


Length of french embeddings: 15177


In [76]:
print('Length of english embeddings:',len(en_embeddings_matrix))


Length of english embeddings: 12026


In [77]:
example_input_batch


<tf.Tensor: shape=(64, 18), dtype=int32, numpy=
array([[  1,   6,   4, ...,   0,   0,   0],
       [  1,  25, 181, ...,   0,   0,   0],
       [  1,   6,  40, ...,   0,   0,   0],
       ...,
       [  1,  34,   3, ...,   0,   0,   0],
       [  1,  99,   4, ...,   0,   0,   0],
       [  1,  61,  84, ...,   0,   0,   0]], dtype=int32)>

In [78]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim,weights=[fr_embeddings_matrix],trainable=True, input_length=max_len)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [79]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 18, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [80]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [81]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 18, 1)


In [82]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim,weights=[en_embeddings_matrix])
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [83]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (64, 12026)


In [84]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [85]:
checkpoint_dir = './training_fr_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [86]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # print("targshape-->",targ.shape)

      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [87]:
EPOCHS = 10

for epoch in range(EPOCHS):
  print("Epoch",epoch)
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 0
Epoch 1 Batch 0 Loss 2.8134
Epoch 1 Batch 100 Loss 1.6838
Epoch 1 Batch 200 Loss 1.4967


InvalidArgumentError: ignored

<h1>TRANSLATE<h1>

In [ ]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)

  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot

In [ ]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

In [ ]:
def translate(sentence):
  result, sentence, attention_plot = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

  attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
  plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))


<h1>Test</h1>

In [ ]:
translate('vehicule')